Credit goes to https://colab.research.google.com/github/count0/colab-gt/blob/master/colab-gt.ipynb for the steps to use graph_tool in Colab.

In [ ]:
!echo "deb http://downloads.skewed.de/apt jammy main" >> /etc/apt/sources.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-matplotlib python3-cairo
!apt purge python3-cairo
!apt install libcairo2-dev pkg-config python3-dev
!pip install --force-reinstall pycairo
!pip install zstandard
!pip install gravis

In [ ]:
from graph_tool.all import *

g = load_graph("HepPh_with_louvain.graphml")

In [ ]:
import random
from tabulate import tabulate
from collections import Counter
import seaborn as sns
random.seed()

def LabelPropagation(G):
    G_copy = G.copy()
    G_copy.set_directed(False)
    comps, hist = label_components(G_copy)
    assert(len(hist) == 1)
    # graph_draw(G_copy)
    labels = {}
    for i in G_copy.iter_vertices():
        labels[i] = [i]

    neighbours = {}
    for i in G_copy.iter_vertices():
        neighbours[i] = G_copy.get_all_neighbours(i)

    vert_order = [v for v in G_copy.iter_vertices()]
    while True:
        tmp_label_map = []
        for i in G_copy.iter_vertices():
            if labels[i][-1] not in tmp_label_map:
                tmp_label_map.append(labels[i][-1])
        print(f"Number of labels: {len(tmp_label_map)}")

        random.shuffle(vert_order)
        for v in vert_order:
            neighbouring_labels = Counter([labels[n][-1] for n in neighbours[v]])
            tie_break = []
            max_occur = neighbouring_labels.most_common(1)[0][1]
            for i in neighbouring_labels.most_common():
                if i[1] == max_occur:
                    tie_break.append(i[0])
                else:
                    break
            labels[v].append(random.choice(tie_break))

        for v in G_copy.iter_vertices():
            neighbouring_labels = Counter([labels[n][-1] for n in neighbours[v]])
            tie_break = []
            max_occur = neighbouring_labels.most_common(1)[0][1]
            for i in neighbouring_labels.most_common():
                if i[1] == max_occur:
                    tie_break.append(i[0])
                else:
                    break
            if labels[v][-1] not in tie_break:
                print(v)
                break
        else:
            break
    comms = G.new_vp("int")
    label_map = {}
    count = 0
    for i in G_copy.iter_vertices():
        if labels[i][-1] not in label_map:
            label_map[labels[i][-1]] = count
            count += 1
        comms[i] = label_map[labels[i][-1]]
    return comms

# g = collection.data["karate"]
part_idxs = LabelPropagation(g)
g.vp.label_comms = part_idxs
# https://stackoverflow.com/questions/12282232/how-do-i-count-occurrence-of-unique-values-inside-a-list
parts = Counter(part_idxs).keys() # equals to list(set(part_idxs))-
freq = Counter(part_idxs).values() # counts the elements' frequency

# https://stackoverflow.com/questions/41140647/python-printing-lists-with-tabulate
table = zip(parts, freq)
print(tabulate(table, headers=["Community Index", "Number of Nodes"]))

with open("comms.txt", "w") as f:
    for i in part_idxs:
        print(i, file=f)

g.save("HepPh_with_louvain_and_label.graphml")

from google.colab import files
files.download('comms.txt')
files.download("HepPh_with_louvain_and_label.graphml")

# https://stackoverflow.com/questions/68875330/how-to-create-a-color-dict-for-multiple-items-from-a-data-frame
# https://seaborn.pydata.org/tutorial/color_palettes.html
# https://seaborn.pydata.org/generated/seaborn.color_palette.html#seaborn.color_palette
# color_map = sns.color_palette("husl", len(freq))

# https://stackoverflow.com/questions/33596264/graph-tool-how-to-change-vertex-colors
# plot_color = g.new_vertex_property('vector<double>')
# g.vertex_properties['plot_color'] = plot_color
# for v in g.vertices():
#     plot_color[v] = color_map[part_idxs[v] % len(freq)]

# graph_draw(g, vertex_fill_color=g.vertex_properties['plot_color'])

Number of labels: 30341
5
Number of labels: 5521
4
Number of labels: 1418
20
Number of labels: 774
131
Number of labels: 573
1
Number of labels: 472
1
Number of labels: 427
27
Number of labels: 399
42
Number of labels: 380
40
Number of labels: 362
26
Number of labels: 341
393
Number of labels: 327
28
Number of labels: 320
142
Number of labels: 310
135
Number of labels: 303
19
Number of labels: 293
81
Number of labels: 290
71
Number of labels: 282
73
Number of labels: 279
159
Number of labels: 278
122
Number of labels: 271
192
Number of labels: 269
449
Number of labels: 265
90
Number of labels: 263
666
Number of labels: 259
695
Number of labels: 255
605
Number of labels: 253
573
Number of labels: 253
639
Number of labels: 251
1126
Number of labels: 249
700
Number of labels: 247
1205
Number of labels: 245
347
Number of labels: 243
1240
Number of labels: 243
4740
Number of labels: 242
4649
Number of labels: 242
3928
Number of labels: 241
4644
Number of labels: 241
689
Number of labels: 24

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>